Let's build a simple neural network to classify images from the FashionMNIST dataset.

**1. Import Libraries**

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

*Checking for GPU Availability*

This code checks if a CUDA-enabled GPU is available and sets the `device` accordingly. If no GPU is available, it defaults to the CPU.

In [2]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
# menit se muze:
# learning rate, batch size, pocet vrstev a pocet neuronu, pocet epoch treninku

# nejvyssi accuracy (86.8 %) se dosahlo pri nastaveni parametru nasledovne:
# BATCH_SIZE = 64, LR = 0.075, EPOCH = 10, OSTATNI SAME

**2. Data Preparation**

In [8]:
# Define a transform to convert images to tensors
transform = transforms.ToTensor()

# Download and load the training data
train_set = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)

# Download and load the test data
test_set = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)

**3. Neural Network Model**

In [9]:
class DenseNet(nn.Module):
    def __init__(self):
        super(DenseNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits

model = DenseNet().to(device)
model

DenseNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

**4. Loss & Optimizer**

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.075)

**5. Training loop**

In [11]:
for epoch in range(10):  # Train for 5 epochs
    running_loss = 0.0
    for images, labels in train_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/10], Loss: {running_loss / len(train_loader):.4f}')

Epoch [1/10], Loss: 0.6560
Epoch [2/10], Loss: 0.4581
Epoch [3/10], Loss: 0.4134
Epoch [4/10], Loss: 0.3856
Epoch [5/10], Loss: 0.3668
Epoch [6/10], Loss: 0.3509
Epoch [7/10], Loss: 0.3378
Epoch [8/10], Loss: 0.3280
Epoch [9/10], Loss: 0.3181
Epoch [10/10], Loss: 0.3081


**6. Evaluation on the test set**

In [12]:
correct = 0
total = 0
with torch.no_grad():  # Disable gradient calculation for evaluation
    for images, labels in test_loader:
        # Move images and labels to the device
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f} %')

Accuracy: 86.79 %


In [ ]:
# PUVODNI PARAMETRY, lr = 0.002, batch size = 256, pocet plne propojenych vrstve 2, prvni vrstva ma 128 neuronu a druha ma 10 neuronu
"""
Epoch [1/5], Loss: 2.2172
Epoch [2/5], Loss: 2.0163
Epoch [3/5], Loss: 1.7878
Epoch [4/5], Loss: 1.5664
Epoch [5/5], Loss: 1.3883
Accuracy: 64.78 %
"""

# BATCH_SIZE = 128, JINAK VSE STEJNE
"""
Epoch [1/5], Loss: 2.0552
Epoch [2/5], Loss: 1.6015
Epoch [3/5], Loss: 1.2802
Epoch [4/5], Loss: 1.0900
Epoch [5/5], Loss: 0.9745
Accuracy: 68.17 %
"""

# BATCH_SIZE = 64, JINAK VSE STEJNE
"""
Epoch [1/5], Loss: 1.9021
Epoch [2/5], Loss: 1.2404
Epoch [3/5], Loss: 0.9650
Epoch [4/5], Loss: 0.8451
Epoch [5/5], Loss: 0.7761
Accuracy: 73.54 %
"""

# BATCH_SIZE = 64,  LR = 0.001, JINAK VSE STEJNE
"""
Epoch [1/5], Loss: 2.0752
Epoch [2/5], Loss: 1.6322
Epoch [3/5], Loss: 1.3069
Epoch [4/5], Loss: 1.1152
Epoch [5/5], Loss: 0.9980
Accuracy: 67.49 %
"""

# BATCH_SIZE = 128, LR = 0.001, JINAK VSE STEJNE
"""
Epoch [1/5], Loss: 2.1878
Epoch [2/5], Loss: 1.9595
Epoch [3/5], Loss: 1.7320
Epoch [4/5], Loss: 1.5268
Epoch [5/5], Loss: 1.3646
Accuracy: 65.06 %
"""

# BATCH_SIZE = 64, LR = 0.002, EPOCH = 10, JINAK SAME
"""
Epoch [1/10], Loss: 1.8997
Epoch [2/10], Loss: 1.2391
Epoch [3/10], Loss: 0.9627
Epoch [4/10], Loss: 0.8408
Epoch [5/10], Loss: 0.7701
Epoch [6/10], Loss: 0.7214
Epoch [7/10], Loss: 0.6845
Epoch [8/10], Loss: 0.6549
Epoch [9/10], Loss: 0.6306
Epoch [10/10], Loss: 0.6102
Accuracy: 78.89 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCHY = 10, JINAK SAME
"""
Epoch [1/10], Loss: 1.4442
Epoch [2/10], Loss: 0.8317
Epoch [3/10], Loss: 0.7027
Epoch [4/10], Loss: 0.6343
Epoch [5/10], Loss: 0.5897
Epoch [6/10], Loss: 0.5585
Epoch [7/10], Loss: 0.5356
Epoch [8/10], Loss: 0.5181
Epoch [9/10], Loss: 0.5039
Epoch [10/10], Loss: 0.4928
Accuracy: 82.01 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCHY = 10, 3 VRSTVY NEURONU, PRVNI VRSTVA 128 NEURONU, DRUHA VRSTVA 64 NEURONU, TRETI VRSTVA 10 NEURONU
"""
Epoch [1/10], Loss: 1.9046
Epoch [2/10], Loss: 1.0124
Epoch [3/10], Loss: 0.7738
Epoch [4/10], Loss: 0.6863
Epoch [5/10], Loss: 0.6297
Epoch [6/10], Loss: 0.5882
Epoch [7/10], Loss: 0.5560
Epoch [8/10], Loss: 0.5315
Epoch [9/10], Loss: 0.5133
Epoch [10/10], Loss: 0.4976
Accuracy: 81.62 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCHY = 5, 3 VRSTVY NEURONU, PRVNI VRSTVA 256 NEURONU, DRUHA 128 NEURONU, TRETI 10
"""
Epoch [1/10], Loss: 1.6290
Epoch [2/10], Loss: 1.6011
Epoch [3/10], Loss: 1.5778
Epoch [4/10], Loss: 1.5571
Epoch [5/10], Loss: 1.5384
Epoch [6/10], Loss: 1.5231
Epoch [7/10], Loss: 1.5105
Epoch [8/10], Loss: 1.5002
Epoch [9/10], Loss: 1.4921
Epoch [10/10], Loss: 1.4845
Accuracy: 48.50 %
"""

# BATCH_SIZE = 64, LR = 0.005, EPOCH = 10, JINAK SAME
"""
Epoch [1/10], Loss: 1.4442
Epoch [2/10], Loss: 0.8317
Epoch [3/10], Loss: 0.7027
Epoch [4/10], Loss: 0.6343
Epoch [5/10], Loss: 0.5897
Epoch [6/10], Loss: 0.5585
Epoch [7/10], Loss: 0.5356
Epoch [8/10], Loss: 0.5181
Epoch [9/10], Loss: 0.5039
Epoch [10/10], Loss: 0.4928
Accuracy: 82.01 %
"""
# BATCH_SIZE = 64, LR = 0.075, EPOCH = 10, OSTATNI SAME
"""
Epoch [1/10], Loss: 0.6560
Epoch [2/10], Loss: 0.4581
Epoch [3/10], Loss: 0.4134
Epoch [4/10], Loss: 0.3856
Epoch [5/10], Loss: 0.3668
Epoch [6/10], Loss: 0.3509
Epoch [7/10], Loss: 0.3378
Epoch [8/10], Loss: 0.3280
Epoch [9/10], Loss: 0.3181
Epoch [10/10], Loss: 0.3081
Accuracy: 86.79 %
"""

